In [1]:
# initial setup
try:
    # settings colab:
    import google.colab
        
except ModuleNotFoundError:    
    # settings local:
    %run "common/0_notebooks_base_setup.py"

default checking
Running command `conda list`... ok
jupyterlab=1.2.6 already installed
pandas=1.0.3 already installed
matplotlib=2.2.2 already installed
bokeh=2.0.0 already installed
seaborn=0.10.0 already installed
ipywidgets=7.5.1 already installed
pytest=5.3.4 already installed
chardet=3.0.4 already installed
psutil=5.7.0 already installed
scipy=1.4.1 already installed
statsmodels=0.11.0 already installed
scikit-learn=0.22.1 already installed
xlrd=1.2.0 already installed
plotly=4.6.0 already installed
chart-studio=1.1.0 already installed
plotly-orca=1.3.1 already installed
Running command `conda install --yes unicodedata2`... ok
Solving environment: ...working... done

# All requested packages already installed.


Importando librerías ...
Las librerías has sido importadas con éxito


In [2]:
data_clean = pd.read_csv("CleanCSV/Clean01_TipoOperacion.csv")

<hr id="Dummies">
<h2 style="color: lightblue">VARIABLES DUMMIE Y SUPERFICIE</h2>

### Generamos una lista de palabras claves obtenidas de la columna 'properati_url' ordenadas por cantidad de repeticiones

In [3]:
pattern = r'http:\/\/www\.properati.com.ar\/\w{6}([\w*-]*)'
pattern_regex = re.compile(pattern)
result_match = data_clean['properati_url'].apply(lambda x: pattern_regex.match(x.lower().replace('-','_')))
data_clean['url_clean'] = result_match.apply(lambda x: x.group(1).replace('_',' '))
result =  result_match.apply(lambda x: x.group(1).split('_'))

list_words = []

def generate_words_series(arr):
    for x in arr:
        list_words.append(str(x))
    return

result.apply(generate_words_series)

serie = pd.Series(list_words)
key_words = serie.value_counts()

In [4]:
display(key_words.head(40))

venta            121119
departamento      71019
garage            56620
lavadero          44034
balcon            42288
propiedades       41507
casa              40537
parrilla          33381
piscina           32148
luminoso          31865
suite             27102
placard           26145
terraza           25101
toilette          22841
subte             21984
linea             21824
patio             21169
vestidor          18011
jardin            17173
quincho           12850
aire              12767
acondicionado     12763
inmobiliaria      12184
sum               11665
dependencias      11316
max               11260
re                11258
cordoba           11195
amenities         10607
baulera           10512
estrenar          10383
vista             10309
de                 9768
san                9556
gimnasio           9363
lujoso             9251
plata              9215
del                9180
rosario            8607
villa              8473
dtype: int64

### a partir de estas palabras claves identificamos las que nos pueden servir para:

### - generar variables dummy:
* garage
* lavadero
* balcon  
* casa
* parrilla
* piscina
* terraza
* patio
* jardin
* quincho
* aire acondicionado
* amenities
* estrenar
* gimnasio

opcionales..
* baulera
* suite
* subte - linea
* vestidor

### - completar datos faltantes
* venta
* departamento


## generamos columnas dummy con la información de la URL


In [5]:
def get_dummy_url_series(pattern):
    pattern_regex = re.compile(pattern)
    result_match = data_clean['url_clean'].apply(lambda x: pattern_regex.search(x))
    result = result_match.apply(lambda x: 1 if x != None else 0)
    result.sum()
    return result

In [6]:
list_columnas_dummy = ['garage|cochera', 'lavadero', 'balcon', 'parrilla', 'piscina', 'terraza', 'patio', 'jardin', 'quincho', 'aire_acondicionado', 'amenities', 'estrenar', 'gimnasio', 'subte', 'baulera']
for column_name in list_columnas_dummy:
    column_values = get_dummy_url_series(column_name)
    data_clean['url_'+ column_name] = column_values

In [7]:
data_clean[data_clean.columns[-16:]] 

,url_clean,url_garage|cochera,url_lavadero,url_balcon,url_parrilla,url_piscina,url_terraza,url_patio,url_jardin,url_quincho,url_aire_acondicionado,url_amenities,url_estrenar,url_gimnasio,url_subte,url_baulera
0,venta ph mataderos lavadero patio inmobiliaria...,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
1,venta departamentos la plata balcon lavadero t...,1,1,1,0,0,0,0,0,0,0,0,1,0,0,0
2,venta departamentos mataderos lavadero placard...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,venta ph liniers patio g goffredo,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4,venta departamentos centro cristina pavone neg...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121215,venta departamento belgrano balcon suite parri...,1,0,1,1,1,0,0,0,1,0,0,0,0,0,0
121216,venta casa beccar suite hidromasaje jardin par...,1,0,0,1,1,0,0,1,1,0,0,0,0,0,0
121217,venta departamento villa urquiza holmberg 2300...,1,1,1,1,0,1,0,0,0,0,1,1,0,0,0
121218,venta departamento plaza colon lavadero lumino...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


## Ahora hacemos lo mismo con la columna 'descripcion'

In [8]:
pattern_desc = r'(\w\w\w\w\w*)'
pattern_desc_regex = re.compile(pattern_desc, flags=re.IGNORECASE )
result_match = data_clean['description'].apply(lambda x: re.finditer(pattern_desc_regex, str(x).lower()))

list_words_desc = []

def generate_words_series_desc(matches):
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            
            cadena = unicodedata.normalize('NFKD', match.group(groupNum)).encode('ASCII', 'ignore') 
            
            list_words_desc.append(cadena)
    return

result =  result_match.apply(generate_words_series_desc)

serie_desc = pd.Series(list_words_desc)
key_words_desc = serie_desc.value_counts()

In [9]:
display(key_words_desc.head(40))

b'cocina'          110088
b'bano'             98520
b'comedor'          96205
b'para'             85190
b'living'           70516
b'dormitorios'      67240
b'ambientes'        56062
b'dormitorio'       53025
b'piso'             50769
b'completo'         49287
b'departamento'     48517
b'balcon'           47086
b'pisos'            45367
b'planta'           44549
b'frente'           43095
b'casa'             39426
b'lavadero'         39128
b'excelente'        37242
b'amplio'           36923
b'edificio'         36296
b'venta'            35989
b'parrilla'         33136
b'cochera'          31810
b'cuenta'           31541
b'placard'          31247
b'suite'            31146
b'sobre'            28899
b'terraza'          28332
b'patio'            25833
b'vista'            24966
b'mesada'           24908
b'gran'             24171
b'propiedad'        23457
b'baja'             23351
b'ubicado'          22884
b'barrio'           22826
b'esta'             22535
b'zona'             22516
b'servicio' 

### Palabras para generar variables dummy
* balcon
* frente
* parrilla
* cochera
* terraza


Opcionales:
suite

### Palabras para completar datos faltantes
* dormitorio/s
* ambientes
* piso - planta
* departamento - casa - edificio
* venta
* ubicado - barrio

### Creamos una funcion que nos permita:

* Observar el contexto de una palabra clave seleccionada dentro de la descripcion
* Obtener un set de indices de filas que en su descripcion contienen esa palabra
* Determinar cuantas filas queremos recorrer

In [10]:
def generate_context_df(word, column, rows=5000, has_context=True, context_words_before=4, context_words_after=4):
    global data_index
    if has_context is True:
        pattern_context = r'(?P<context>(\w*\W?\s\W?){0,' + str(context_words_before) + '}' + word + '(\w*\W?\s\W?){0,' + str(context_words_after) + '})'
    else:    
        pattern_context = word

    pattern_context_regex = re.compile(pattern_context, flags=re.IGNORECASE )
    result_match = data_clean[column].apply(lambda x: re.finditer(pattern_context_regex, str(x).lower()))

    list_words_desc_context = []
    list_words_desc_index = []
    data_index = 0
    df_context = pd.DataFrame()
        
    def generate_context_df(matches):
        global data_index
        for matchNum, match in enumerate(matches, start=1):  
            if has_context is True:
                context = match.group('context')     
                list_words_desc_context.append(context)
            list_words_desc_index.append(data_index)
        data_index += 1
        return
    
    if rows == 0:
        result = result_match.apply(generate_context_df)
    else:
        result = result_match[0:rows].apply(generate_context_df)
        
    if has_context is True:
        df_context['context'] = list_words_desc_context
        
    df_context['index'] = list_words_desc_index
    print('cantidad de matches: ', df_context.shape[0], ' en ', rows if rows != 0 else 'TODOS los' , ' registros buscados')
    return df_context

### Comprobamos su funcionamiento solicitando 5000 registros de la potencial variable dummy 'garage|cochera'

In [11]:
generate_context_df("garage|cochera", "description", 5000, True).head(10)

cantidad de matches:  2726  en  5000  registros buscados


,context,index
0,cochera semicubierta.,1
1,cochera,10
2,"cochera y jardín propio,",15
3,cochera fija subterranea y,17
4,cochera. en cumplimiento de,23
5,cochera. entrega marzo,24
6,cochera,24
7,cochera. entrega marzo,25
8,cochera,25
9,cochera. entrega marzo,28


### Creamos una funcion que nos permite generar variables dummies a partir de palabras clave

In [12]:
def create_dummy(data, data_dummy, column_name):
    data_dummy.drop_duplicates(inplace=True)
    col_dummy_name = 'desc_' + column_name.replace(' ','_')
    data_dummy[col_dummy_name] = 1
    data_dummy[col_dummy_name].astype(int)
    data_clean_dummy = pd.merge(data, data_dummy, how='left', left_on='Unnamed: 0', right_on='index')
    del data_clean_dummy['index']
    data_clean_dummy[col_dummy_name].fillna(0, inplace=True)
    print('Col generada: '  + col_dummy_name + ' ' + str(data_dummy.shape[0]))
    return data_clean_dummy

### Creamos una lista de palabras claves que recorremos utilizando la funcion mencionada anteriormente

como resultado vamos a haber agregado a nuestro DataFrame una columna por cada variable dummy

In [13]:
for columna in list_columnas_dummy:
    data_dummy = generate_context_df(columna, "description", 0, False)
    data_clean = create_dummy(data_clean, data_dummy, columna)

cantidad de matches:  63489  en  TODOS los  registros buscados
Col generada: desc_garage|cochera 50494
cantidad de matches:  40637  en  TODOS los  registros buscados
Col generada: desc_lavadero 38329
cantidad de matches:  19817  en  TODOS los  registros buscados
Col generada: desc_balcon 16577
cantidad de matches:  38015  en  TODOS los  registros buscados
Col generada: desc_parrilla 34037
cantidad de matches:  18059  en  TODOS los  registros buscados
Col generada: desc_piscina 15247
cantidad de matches:  35678  en  TODOS los  registros buscados
Col generada: desc_terraza 27509
cantidad de matches:  27922  en  TODOS los  registros buscados
Col generada: desc_patio 21494
cantidad de matches:  9955  en  TODOS los  registros buscados
Col generada: desc_jardin 7785
cantidad de matches:  14636  en  TODOS los  registros buscados
Col generada: desc_quincho 13094
cantidad de matches:  0  en  TODOS los  registros buscados
Col generada: desc_aire_acondicionado 0
cantidad de matches:  12381  en  T

In [14]:
data_clean[data_clean.columns[-16:]].head(3)

,url_baulera,desc_garage|cochera,desc_lavadero,desc_balcon,desc_parrilla,desc_piscina,desc_terraza,desc_patio,desc_jardin,desc_quincho,desc_aire_acondicionado,desc_amenities,desc_estrenar,desc_gimnasio,desc_subte,desc_baulera
0,0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## corroboramos la relacion entre variables dummy obtenidas con title y description

recorremos las columnas buscando el valor de correlación

In [15]:
for col in list_columnas_dummy:
    col_name = col.replace(' ','_')
    print (col + ': ', data_clean[['desc_' + col_name ,'url_' + col_name]].corr().iloc[1,0])
    

garage|cochera:  0.8602882129317675
lavadero:  0.8681289024952435
balcon:  0.5081233097817958
parrilla:  0.9822555938926155
piscina:  0.6067186979410569
terraza:  0.938134060364859
patio:  0.9847387877679887
jardin:  0.6237720897463092
quincho:  0.9830049594035731
aire_acondicionado:  nan
amenities:  0.9760022603551674
estrenar:  0.9747240900685272
gimnasio:  0.8207016867870898
subte:  0.3481246803358617
baulera:  0.972369268850563


In [16]:
corr_columns = [ [col, round(data_clean[['desc_' + col.replace(' ','_') ,'url_' + col.replace(' ','_')]].corr().iloc[1,0],2)] for col in list_columnas_dummy]
corr_columns

[['garage|cochera', 0.86],
 ['lavadero', 0.87],
 ['balcon', 0.51],
 ['parrilla', 0.98],
 ['piscina', 0.61],
 ['terraza', 0.94],
 ['patio', 0.98],
 ['jardin', 0.62],
 ['quincho', 0.98],
 ['aire_acondicionado', nan],
 ['amenities', 0.98],
 ['estrenar', 0.97],
 ['gimnasio', 0.82],
 ['subte', 0.35],
 ['baulera', 0.97]]

.

ahora recorremos las columnas buscando la diferencia porcentual de aciertos

In [17]:
for col in list_columnas_dummy:
    col_name = col.replace(' ','_')
    serie = data_clean['desc_' + col_name ] == data_clean['url_' + col_name]
    print(col + ': ',round(1 - (serie.shape[0] - serie.sum()) / serie.shape[0],2))
    

garage|cochera:  0.93
lavadero:  0.94
balcon:  0.78
parrilla:  0.99
piscina:  0.85
terraza:  0.98
patio:  1.0
jardin:  0.92
quincho:  1.0
aire_acondicionado:  1.0
amenities:  1.0
estrenar:  1.0
gimnasio:  0.98
subte:  0.89
baulera:  1.0


## Seleccion y refinamiento en variables dummies

In [18]:
#unificamos los valores encontrados en descripcion y URL
def unify_cols(column, column_name=""):
    if column_name == "":
        column_name = column
    data_clean[column_name] = data_clean.apply(lambda df: max([df['desc_' + column],df['url_' + column]]), axis=1)
    del data_clean['desc_' + column]
    del data_clean['url_' + column]
    print(column_name, data_clean[column_name].sum())

In [19]:
unify_cols('garage|cochera', 'garage')
unify_cols('lavadero')
unify_cols('balcon')
unify_cols('parrilla')
unify_cols('piscina')
unify_cols('jardin')
unify_cols('patio')
unify_cols('amenities')
unify_cols('estrenar')
unify_cols('gimnasio')
unify_cols('subte')
unify_cols('baulera')


garage 57880.0
lavadero 44906.0
balcon 42994.0
parrilla 34143.0
piscina 32584.0
jardin 17364.0
patio 21600.0
amenities 11026.0
estrenar 10739.0
gimnasio 9633.0
subte 17893.0
baulera 10923.0


In [20]:
del data_clean['desc_aire_acondicionado']
del data_clean['url_aire_acondicionado']

In [21]:
data_clean['piscina_house'] = 0
data_clean['piscina_depart'] = 0
piscina_house_mask = (data_clean['property_type'] == 'house') & (data_clean['piscina'] == 1)
piscina_not_house_mask = (data_clean['property_type'] != 'house') & (data_clean['piscina'] == 1)
data_clean.loc[piscina_house_mask, 'piscina_house'] = 1
data_clean.loc[piscina_not_house_mask, 'piscina_depart'] = 1
del data_clean['piscina']

In [22]:
print(data_clean['piscina_house'].sum())
print(data_clean['piscina_depart'].sum())

15380
17204


## Imputación de ambientes a partir de la columna 'titles'

In [23]:
df_cant_ambientes = generate_context_df("ambiente[s]|monoambiente", "title", 0, True, 1,0)

cantidad de matches:  17363  en  TODOS los  registros buscados


In [24]:
df_cant_ambientes.context.value_counts().head(10)

2 ambientes         4384
3 ambientes         4142
monoambiente        3338
4 ambientes         2300
dos ambientes        918
5 ambientes          697
tres ambientes       477
venta. ambientes     437
6 ambientes          194
cuatro ambientes     109
Name: context, dtype: int64

In [25]:
df_cant_ambientes.context = df_cant_ambientes.context.str.replace(' ambientes','')
df_cant_ambientes.context.value_counts().head(10)

2               4384
3               4142
monoambiente    3338
4               2300
dos              918
5                697
tres             477
venta.           437
6                194
cuatro           109
Name: context, dtype: int64

In [26]:
import re

dic = {"dos": "2", "tres": "3", "cuatro": "4", "cinco": "5", "seis": "6", "siete": "7", "ocho": "8", "monoambiente": "1"} # define desired replacements here

def replace_all(text):
    for key in dic:
        text = text.replace(key, dic[key])
    return text

df_cant_ambientes.context = df_cant_ambientes.context.apply(replace_all)
df_cant_ambientes.context.value_counts().head(10)

2            5302
3            4619
1            3353
4            2409
5             730
venta.        437
6             198
7              69
ambientes      47
amplios        26
Name: context, dtype: int64

In [27]:
df_cant_ambientes_not_numeric_mask = df_cant_ambientes.context.apply(lambda x: False if x.isnumeric() else True)
df_cant_ambientes.drop(df_cant_ambientes[df_cant_ambientes_not_numeric_mask].index,inplace=True)
df_cant_ambientes.drop(df_cant_ambientes[df_cant_ambientes.context == '4000'].index,inplace=True)
df_cant_ambientes.context = df_cant_ambientes.context.astype(int)

In [28]:
df_cant_ambientes.rename({'context': 'desc_ambientes'}, axis=1,inplace=True)
df_cant_ambientes

,desc_ambientes,index
0,1,7
1,2,8
2,2,12
3,3,17
4,4,21
...,...,...
17358,3,121112
17359,2,121138
17360,3,121139
17361,3,121153


In [29]:
print(df_cant_ambientes.shape)
df_cant_ambientes.drop_duplicates(['index'], inplace=True)
print(df_cant_ambientes.shape)

(16716, 2)
(16658, 2)


In [30]:
data_clean_ambientes = pd.merge(data_clean, df_cant_ambientes, how='left', left_on='Unnamed: 0', right_on='index')
del data_clean_ambientes['index']
data_clean = data_clean_ambientes

In [31]:
#Comprobamos la correlación entre los datos obtenidos por la descripciones y los preexistentes en data_clean
#encontramos que la correlacion entre ambas variables es 0,91

data_clean_ambientes_mask = data_clean.rooms.notnull() & data_clean.desc_ambientes.notnull()
data_clean.loc[data_clean_ambientes_mask,['rooms','desc_ambientes']].corr()

,rooms,desc_ambientes
rooms,1.000000,0.916597
desc_ambientes,0.916597,1.000000


In [32]:
#creamos una mascara para imputar datos faltantes en rooms
data_clean_ambientes_mask = data_clean.rooms.isnull() & data_clean.desc_ambientes.notnull()
data_clean_ambientes_mask.sum()

7779

In [33]:
#imputamos los datos 
data_clean.loc[data_clean_ambientes_mask,['rooms']] = data_clean.loc[data_clean_ambientes_mask,['desc_ambientes']].desc_ambientes


In [34]:
#verificamos que los nulos se hayan reemplazado por valores de la columna desc_ambientes
data_clean_ambientes_mask_2 = data_clean.rooms.isnull() & data_clean.desc_ambientes.notnull()
data_clean_ambientes_mask_2.sum()

0

In [35]:
#eliminamos la columna desc_ambientes
del data_clean['desc_ambientes']

## imputacion de ambientes a partir de baños

In [36]:
banios = generate_context_df("baños|baño", "description", 0, True, 1,0)

cantidad de matches:  121131  en  TODOS los  registros buscados


In [37]:
banios.context.value_counts().head(20)

baño              100496
2 baños             5567
dos baños           3071
en baños            1663
3 baños             1368
y baños             1187
baños                919
de baños             852
los baños            710
4 baños              401
- baños              322
con baños            300
tres baños           289
* baños              146
5 baños              134
1 baños               95
placard, baños        93
cocinas, baños        88
cocina, baños         83
ambos baños           74
Name: context, dtype: int64

In [38]:
dic = {"2 baños": "2", "dos baños": "2", "3 baños": "3", "4 baños": "4", "tres baños": "3", "5 baños": "5", "1 baños": "1", "ambos baños": "2", "baño": "1"}

def replace_all(text):
    for key in dic:
        text = text.strip().replace(key, dic[key])
    return text
banios_replaced = banios
banios_replaced.context = banios.context.apply(replace_all)

In [39]:
banios_replaced.context.value_counts().head(10)

1         100591
2           8712
en 1s       1663
3           1657
y 1s        1187
1s           969
de 1s        852
los 1s       710
4            401
- 1s         322
Name: context, dtype: int64

In [40]:
banios_replaced_not_numeric_mask = banios_replaced.context.apply(lambda x: False if x.isnumeric() else True)

In [41]:
banios_replaced.drop(banios_replaced[banios_replaced_not_numeric_mask].index,inplace=True)

In [42]:
banios_replaced.context = banios_replaced.context.astype(int)

In [43]:
banios_replaced_outlayers_mask = banios_replaced.context > 5
banios_replaced.drop(banios_replaced[banios_replaced_outlayers_mask].index,inplace=True)

In [44]:
banios_replaced.context.value_counts().head(10)

1    100591
2      8713
3      1657
4       401
5       134
Name: context, dtype: int64

In [45]:
banios_replaced.drop_duplicates(['index'], inplace=True)

In [46]:
banios_replaced.rename({'context': 'desc_banios'}, axis=1,inplace=True)
banios_replaced

,desc_banios,index
0,1,0
1,1,1
2,1,2
3,1,4
4,1,5
...,...,...
121121,2,121214
121122,1,121216
121126,1,121217
121127,1,121218


In [47]:
data_clean_banios = pd.merge(data_clean, banios_replaced, how='left', left_on='Unnamed: 0', right_on='index')
del data_clean_banios['index']
data_clean = data_clean_banios

In [48]:
banios_not_null_mask = data_clean.desc_banios.notnull() & data_clean.rooms.notnull()

In [49]:
#analizamos la correlación entre habitaciones y baños
feature_cols = ['desc_banios', 'rooms']
data_clean.loc[banios_not_null_mask, feature_cols].corr()

,desc_banios,rooms
desc_banios,1.000000,0.254149
rooms,0.254149,1.000000


In [50]:
corr_banios_rooms = data_clean.loc[banios_not_null_mask, feature_cols].apply(lambda df: str(df['desc_banios'].astype(int)) + '-' + str(df['rooms'].astype(int)), axis=1)

In [51]:
corr_banios_rooms.value_counts().head(10)

1-3    8732
1-2    8695
1-4    5405
1-1    4428
1-5    2413
2-4    1112
1-6     963
2-3     898
1-7     503
2-5     486
dtype: int64

### se desestima la imputacion de habitaciones a partir de la cantidad de baños por su baja correlación

### Ahora vamos a trabajar sobre la columna 'descripcion' para:

* capturar los valores en m2 de las descripciones donde existan faltantes
* obervar el contexto de los valores capturados
* preservar el indice para una posterior imputación de datos
* crear una lista de palabras claves a buscar ordenadas por cantidad de repeticiones

In [52]:
#tomamos solo las filas con faltantes en los campos de m2
#normalizamos las cadenas de texto para remover caracteres especiales
mask_m2_any_null = data_clean.surface_total_in_m2.isnull() | data_clean.surface_covered_in_m2.isnull()
data_m2_null = data_clean.loc[mask_m2_any_null, 'description'].apply(lambda x: unicodedata.normalize('NFKD', str(x)).encode('utf-8', 'ignore').lower())

#generamos un patron que captura el valor de los m2 y el contexto en el que fue encontrado
pattern_m2 = r'(?P<before>(\w*\W?\s\W?){0,4})\s?(?P<value>\d{1,5})\s*(m2|metros)\s?(?P<after>(\W?\w*\W?\s?){0,4})'

pattern_m2_regex = re.compile(pattern_m2, flags=re.IGNORECASE )
result_match = data_m2_null.apply(lambda x: re.finditer(pattern_m2_regex, str(x)))

df_result_match = result_match.to_frame().copy()
df_result_match['indice'] = result_match.to_frame().index
df_result_match

list_words_desc_after = []
list_words_desc_before = []
list_words_desc_values = []
list_words_desc_index = []

df_m2 = pd.DataFrame(columns=('before', 'm2', 'after', 'index'))

#recorremos los resultados
for index, row in df_result_match.iterrows():
    matches = row['description']
    indice = row['indice']
    for match in matches:  
        before = match.group('before')
        after =  match.group('after')
        value = match.group('value').strip()    

        list_words_desc_before.append(before)
        list_words_desc_values.append(value)
        list_words_desc_index.append(indice)
        list_words_desc_after.append(after)

df_m2['m2'] = list_words_desc_values
df_m2['before'] = list_words_desc_before
df_m2['index'] = list_words_desc_index
df_m2['after'] = list_words_desc_after
display(df_m2.head(20))
print('cantidad de matches: ', df_m2.size)

,before,m2,after,index
0,y servicios. lote de,1514,.,10
1,,23,contado u$d250.000haus,15
2,x83o de,50,"cuadrados,con terraza y",65
3,balcon. super luminoso tiene,38,aprox. cuenta con ban\,66
4,monoambientes de,22,+ 5.30m21 dormitorio,101
5,al frente de,36,+ 5.30m21 dormitorio,101
6,,50,+ 2.30m22 dormitorios,101
7,,50,+ 2.70m2entrega enero,101
8,monoambientes de,22,+ 5.30m21 dormitorio,102
9,al frente de,36,+ 5.30m21 dormitorio,102


cantidad de matches:  75476


### Generamos una lista de palabras claves tomadas del contexto del DataFrame anterior
### Estas palabras nos permitirán, luego, imputar datos faltantes en m2

In [53]:
keys_m2 = list_words_desc_before + list_words_desc_after
list_keys_words_m2 = []
for key in keys_m2:
    list_key_words = key.replace('.', '').replace(',', '').replace(':', '').split(' ')
    for word in list_key_words:
        if (word != '') & (len(word) > 2):
            list_keys_words_m2.append(word)
        
list_keys_words_m2

serie_keys_words_m2 = pd.Series(list_keys_words_m2)
serie_keys_words_m2_counts = serie_keys_words_m2.value_counts()
serie_keys_words_m2_counts.head(30)

con             3685
superficie      2197
cubiertos       1582
lote            1458
total           1192
cubierta        1191
sobre           1180
terreno         1154
una              956
sup              802
frente           760
casa             719
planta           712
x81n             704
aprox            685
ambientes        669
totales          596
dos              585
por              578
del              575
cuadrados        565
departamento     541
los              534
terraza          493
cocina           471
piso             470
cuenta           452
dormitorios      452
living           451
comedor          410
dtype: int64

### Elegimos de la lista las palabras mas relevantes
### Creamos a partir de ellas dos series con los cuales imputaremos valores en las columnas 'surface_total_in_m2' y 'surface_covered_in_m2'

In [54]:
#Unimos las columnas de contexto
df_m2['context'] = df_m2['before'] + " " + df_m2['after']

#buscamos los m2 totales
WORD = "total" #buscamos los m2 totales

key_word_mask_totales = df_m2['context'].apply(lambda x: True if WORD in x else False)
print('cantidad de veces que', WORD.upper(), 'fué encontrada: ', key_word_mask_totales.sum())

#df_m2_totales = df_m2.loc[key_word_mask,["m2","index"]]
df_m2_totales = df_m2.loc[key_word_mask_totales,["m2","index"]]

#buscamos los m2 cubiertos
list_words = ["sup cub", "cubiert"] 

def find_list_words_m2(list, row):
    existe = False
    for word in list:
        if word in row:
            existe = True
    return existe

key_word_mask_cubiertos = df_m2['context'].apply(lambda x : find_list_words_m2(list_words, x))
print('cantidad de veces que *sup cub* y *cubiert* fueron encontradas: ', key_word_mask_cubiertos.sum())

df_m2_cubiertos = df_m2.loc[key_word_mask_cubiertos,["m2","index", "context"]]

cantidad de veces que TOTAL fué encontrada:  1969
cantidad de veces que *sup cub* y *cubiert* fueron encontradas:  3641


In [55]:
m2_totales_a_imputar = df_m2_totales.groupby('index').max()
m2_totales_a_imputar

,m2
index,
175,88
491,122
538,2290
539,2290
869,84
...,...
120585,110
120633,56
120634,655


### Reemplazamos los valores obtenidos para 'm2 totales'

In [56]:
print("\n ..data_clean.loc[m2_totales_a_imputar.index, 'surface_total_in_m2']:")
display(data_clean.loc[m2_totales_a_imputar.index, 'surface_total_in_m2']) 
# ALGUNOS YA TIENE VALORES, NO LOS VAMOS A SOBREESCRIBIR.
mask_m2_total_imputar_null = data_clean.loc[m2_totales_a_imputar.index, 'surface_total_in_m2'].isnull()
mask_m2_total_imputar_null_final= mask_m2_total_imputar_null[mask_m2_total_imputar_null]   
# SOLO VAMOS A IMPUTAR DONDE NO TENGO NINGÚN VALOR
print("\n.. data_clean.loc[mask_m2_total_imputar_null_final.index, 'surface_total_in_m2']:")
data_clean.loc[mask_m2_total_imputar_null_final.index, 'surface_total_in_m2']


 ..data_clean.loc[m2_totales_a_imputar.index, 'surface_total_in_m2']:


index
175         NaN
491         NaN
538         NaN
539         NaN
869         NaN
          ...  
120585      NaN
120633      NaN
120634      NaN
120722    116.0
120929    287.0
Name: surface_total_in_m2, Length: 1827, dtype: float64


.. data_clean.loc[mask_m2_total_imputar_null_final.index, 'surface_total_in_m2']:


index
175      NaN
491      NaN
538      NaN
539      NaN
869      NaN
          ..
120184   NaN
120527   NaN
120585   NaN
120633   NaN
120634   NaN
Name: surface_total_in_m2, Length: 1549, dtype: float64

In [57]:
print('\n ..data_clean:')
display(data_clean.loc[mask_m2_total_imputar_null_final.index, 'surface_total_in_m2'])
data_clean.loc[mask_m2_total_imputar_null_final.index, 'surface_total_in_m2'] = \
                                                m2_totales_a_imputar.loc[mask_m2_total_imputar_null_final.index, 'm2']

print('\n Luego de imputarlos..data_clean:')
display(data_clean.loc[mask_m2_total_imputar_null_final.index, 'surface_total_in_m2'])

print('\n m2_totales_a_imputar:')
m2_totales_a_imputar.loc[mask_m2_total_imputar_null_final.index, 'm2']


 ..data_clean:


index
175      NaN
491      NaN
538      NaN
539      NaN
869      NaN
          ..
120184   NaN
120527   NaN
120585   NaN
120633   NaN
120634   NaN
Name: surface_total_in_m2, Length: 1549, dtype: float64


 Luego de imputarlos..data_clean:


index
175         88
491        122
538       2290
539       2290
869         84
          ... 
120184      95
120527      89
120585     110
120633      56
120634     655
Name: surface_total_in_m2, Length: 1549, dtype: object


 m2_totales_a_imputar:


index
175         88
491        122
538       2290
539       2290
869         84
          ... 
120184      95
120527      89
120585     110
120633      56
120634     655
Name: m2, Length: 1549, dtype: object

### Pequeña verificación de la asignación

In [58]:
print('\n ..data_clean:')
display(data_clean.loc[537:540, 'surface_total_in_m2'])
print('\n ..m2_totales_a_imputar:')
display(m2_totales_a_imputar.loc[537:540, 'm2'])


 ..data_clean:


537     NaN
538    2290
539    2290
540     NaN
Name: surface_total_in_m2, dtype: object


 ..m2_totales_a_imputar:


index
538    2290
539    2290
Name: m2, dtype: object

In [59]:
correccionSurfaceTotalInM2 = data_clean.surface_total_in_m2.apply(lambda x: str(x).lower().replace("m2", "").replace("superficie", "").strip())
correccionSurfaceTotalInM2Final = correccionSurfaceTotalInM2.apply(lambda x: x if x[0] != "." else x[1:]).astype(np.float64)
data_clean.loc[correccionSurfaceTotalInM2Final.index, "surface_total_in_m2"] = correccionSurfaceTotalInM2Final

In [60]:
property_type_dummies = pd.get_dummies(data_clean["property_type"], columns=["property_type"], prefix="tipo_propiedad", drop_first=True)
data_clean = pd.concat([data_clean, property_type_dummies], axis=1)

In [61]:
data_clean.drop(
    ['Unnamed: 0'], \
    axis=1, inplace=True)

In [62]:
data_clean.to_csv("CleanCSV/Clean02_DummiesYSuperficie.csv")